In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/ColabData/cran

/content/drive/My Drive/ColabData/cran


In [ ]:
import pandas as pd
import os
import random
import string
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint
# Libraries for visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [326]:
# Load the Cranfield dataset files into Python objects
with open('cran.all.1400') as f:
    cran_docs = f.read().splitlines()

with open('cran.qry') as f:
    cran_queries = f.read().splitlines()

with open('cranqrel') as f:
    cran_relevance = [line.split() for line in f.read().splitlines()]

# Convert the relevance list into a pandas DataFrame
cran_relevance = pd.DataFrame(cran_relevance, columns=['query_id', 'doc_id', 'relevance'])

# print(cran_docs)
# print(cran_queries)
print(len(cran_relevance))
print(len(cran_relevance[cran_relevance['relevance']>'1']))


1837
1484


In [328]:
def load_data():
  # read data from CISI.ALL file and store in dictinary
  # Open the file for reading
  with open('cran.all.1400', 'r') as f:
      data = f.read().splitlines()

  # Initialize an empty dictionary
  doc_dict = {}

  # Initialize variables to store the current document ID and text
  current_id = None
  current_text = ''

  # Loop over the list of strings
  for line in data:
      if line.startswith('.I'):
          # If we've already seen a document ID, add the current text to the doc_dict
          if current_id is not None:
              doc_dict[current_id] = current_text.strip()
          # Reset the current text and update the current ID
          current_text = ''
          current_id = line.split()[1]
      elif line.startswith('.T'):
          # Remove the ".T" tag
          line = line[2:]
          # Append the line to the current text
          current_text += line
      else:
          # Append the line to the current text
          current_text += line

  # Add the last document to the doc_dict
  if current_id is not None:
      doc_dict[current_id] = current_text.strip()

# Print the resulting doc_dict
# print(doc_dict)


  # Open the file for reading
  with open('cran.qry', 'r') as f:
      data = f.read().splitlines()

  # Initialize an empty dictionary
  qry_dict = {}

  # Loop over the list of strings
  for i in range(len(data)):
      if data[i].startswith('.I'):
          # Get the ID number by splitting the string and taking the second element
          id_num = data[i].split()[1]
      elif data[i].startswith('.W'):
          # Join the remaining sentences into a single string
          sentence = ' '.join(data[i+1:i+3])
          # Add the sentence to the dictionary with the ID number as the key
          qry_dict[id_num] = sentence
  qry_dict = {str(int(k)): v for k, v in qry_dict.items()}

  

  # read data from CISI.REL file and store in dictinary
  rel_dict = {}
  
  with open('cranqrel') as f:
    doc_rel, qry_rel = set(), set()
    for l in f.readlines():
      qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
      doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[1]
      relevance = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[2]
      if relevance > '1':
        qry_rel.add(qry_id)
        doc_rel.add(doc_id)

        if qry_id in rel_dict:
          rel_dict[qry_id].append(doc_id)
        else:
          rel_dict[qry_id] = []
          rel_dict[qry_id].append(doc_id)

  
  doc_dict = {int(id): doc for (id, doc) in doc_dict.items() if id in doc_rel}
  qry_dict = {int(id): qry for (id, qry) in qry_dict.items() if id in qry_rel}
  rel_dict = {int(qid): list(map(int, did_lst)) for (qid, did_lst) in rel_dict.items()}
  rel_dict = {k: v for k, v in rel_dict.items() if k in qry_dict}


  return doc_dict, qry_dict, rel_dict

In [329]:
doc_dict, qry_dict, rel_dict = load_data()



In [ ]:
doc_dict.keys()

In [331]:
rel_sum = 0
for doc_ids in rel_dict.values():
  rel_sum += len(doc_ids)
print(f'Number of queries = {len(qry_dict)}')
print(f'* Example: {qry_dict[1]}')
print(f'\nNumber of documents = {len(doc_dict)}')
print(f'* Example: {doc_dict[2]}')
print(f'\nNumber of q-d+ = {rel_sum}')
print(f'* Example:', {1: rel_dict[1]})

Number of queries = 143
* Example: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .

Number of documents = 794
* Example: simple shear flow past a flat plate in an incompressible fluid of smallviscosity ..Ating-yili.Bdepartment of aeronautical engineering, rensselaer polytechnicinstitutetroy, n.y..Wsimple shear flow past a flat plate in an incompressible fluid of smallviscosity .in the study of high-speed viscous flow past a two-dimensional body itis usually necessary to consider a curved shock wave emitting from thenose or leading edge of the body .  consequently, there exists aninviscid rotational flow region between the shock wave and the boundarylayer .  such a situation arises, for instance, in the study of thehypersonic viscous flow past a flat plate .  the situation is somewhatdifferent from prandtl's classical boundary-layer problem . in prandtl'soriginal problem the inviscid free stream outside the boundary layer isirrota

In [ ]:
rel_sum = 0
for doc_ids in rel_dict.values():
  rel_sum += len(doc_ids)
print(f'Number of queries = {len(qry_dict)}')
print(f'* Example: {qry_dict[1]}')
print(f'\nNumber of documents = {len(doc_dict)}')
print(f'* Example: {doc_dict[2]}')
print(f'\nNumber of q-d+ = {rel_sum}')
print(f'* Example:', {1: rel_dict[1]})

Number of queries = 152
* Example: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .

Number of documents = 924
* Example: simple shear flow past a flat plate in an incompressible fluid of smallviscosity ..Ating-yili.Bdepartment of aeronautical engineering, rensselaer polytechnicinstitutetroy, n.y..Wsimple shear flow past a flat plate in an incompressible fluid of smallviscosity .in the study of high-speed viscous flow past a two-dimensional body itis usually necessary to consider a curved shock wave emitting from thenose or leading edge of the body .  consequently, there exists aninviscid rotational flow region between the shock wave and the boundarylayer .  such a situation arises, for instance, in the study of thehypersonic viscous flow past a flat plate .  the situation is somewhatdifferent from prandtl's classical boundary-layer problem . in prandtl'soriginal problem the inviscid free stream outside the boundary layer isirrota

In [ ]:
qry_dict.keys()

In [ ]:
rel_dict

Preprocess Text

In [332]:
# remove all punctuations
for k, v in qry_dict.items():
  qry_dict[k] = v.translate(str.maketrans('', '', string.punctuation))
for k, v in doc_dict.items():
  doc_dict[k] = v.translate(str.maketrans('', '', string.punctuation))

In [333]:
print('Query')
print(f'* Example: {qry_dict[105]}')
print('Document')
print(f'* Example: {doc_dict[13]}')

Query
* Example: cant the static deflection shapes be used in predicting flutter in place of vibrational shapes  if so  can we provide a justification by
Document
* Example: similarity laws for stressing heated wings AtsienhsBj ae scs 20 1953 1Wsimilarity laws for stressing heated wings   it will be shown that the differential equations for a heatedplate with large temperature gradient and for a similar plate atconstant temperature can be made the same by a propermodification of the thickness and the loading for the isothermal plate this fact leads to the result that the stresses in the heated platecan be calculated from measured strains on the unheated plate bya series of relations called the similarity laws   theapplication of this analog theory to solid wings under aerodynamicheating is discussed in detail   the loading on the unheated analogwing is however complicated and involves the novel conceptof feedback and body force loading   the problem of stressinga heated boxwing structu

In [336]:
random.seed(500)
unrel_dict = {}
for q, doc_ids in rel_dict.items():
  unrel = [d for d in doc_dict.keys() if d not in doc_ids]
  random.shuffle(unrel)
  unrel_dict[q] = unrel

In [337]:
print(f'* Example:', {1: unrel_dict[1]})

* Example: {1: [996, 188, 1087, 293, 842, 891, 687, 846, 259, 466, 1134, 249, 683, 593, 305, 1077, 438, 623, 1350, 657, 567, 748, 799, 478, 809, 1076, 302, 720, 574, 1397, 734, 727, 604, 236, 668, 962, 829, 615, 1051, 408, 678, 913, 821, 1150, 571, 914, 62, 777, 257, 1138, 1228, 1157, 1159, 1048, 931, 606, 1018, 785, 594, 1042, 559, 1024, 79, 589, 603, 326, 883, 843, 487, 1392, 1297, 1034, 934, 923, 666, 172, 1257, 834, 23, 48, 807, 775, 932, 794, 989, 1136, 267, 310, 164, 420, 1001, 811, 291, 1035, 196, 1109, 1100, 1101, 797, 1103, 1394, 1356, 458, 602, 1326, 1058, 598, 1111, 1154, 84, 618, 911, 388, 952, 316, 1161, 1188, 1151, 55, 1213, 616, 315, 841, 422, 122, 978, 823, 261, 1019, 1046, 1300, 1131, 294, 1177, 810, 440, 1305, 864, 723, 1304, 1105, 865, 969, 840, 180, 394, 917, 5, 845, 1365, 16, 202, 1106, 776, 1214, 332, 1370, 731, 881, 1302, 564, 46, 769, 163, 437, 205, 277, 311, 215, 1379, 415, 1112, 393, 460, 730, 577, 801, 788, 121, 1107, 725, 276, 44, 481, 105, 25, 585, 169, 125

In [338]:
test_split = int(len(rel_dict) * 0.1)
print(f'Number of queries in train set: {len(rel_dict) - test_split}')
print(f'Number of queries in validation set: {test_split}')

Number of queries in train set: 129
Number of queries in validation set: 14


In [339]:
random.seed(500)
test_qry_set = random.sample(list(rel_dict.keys()), test_split)

In [341]:

print('Queries in validation set:')
print(test_qry_set)

Queries in validation set:
[187, 213, 189, 108, 150, 97, 54, 136, 50, 9, 93, 133, 102, 156]


In [342]:
word_hashing = CountVectorizer(lowercase=True, analyzer='char_wb', ngram_range=(3,3))
word_hashing.fit(list(doc_dict.values()) + list(qry_dict.values()))

CountVectorizer(analyzer='char_wb', ngram_range=(3, 3))

In [343]:
wh_size = len(word_hashing.vocabulary_)
print(f'Number of letter trigrams: {wh_size}')
print(word_hashing.get_feature_names_out())

Number of letter trigrams: 8501
[' 0 ' ' 00' ' 01' ... 'zur' 'zz ' 'zzl']


In [344]:
train_set = []
for q, doc_ids in rel_dict.items():
  if q not in test_qry_set:
    for idx, d in enumerate(doc_ids):
      sample = [q, d] + unrel_dict[q][idx * 4:(idx + 1) * 4]
      train_set.append(sample)
train_set = np.array(train_set)

In [345]:
print('Examples of training samples in train set: [q, d+, d1-, d2-, d3-, d4-]')
pprint(train_set)

Examples of training samples in train set: [q, d+, d1-, d2-, d3-, d4-]
array([[   1,  184,  996,  188, 1087,  293],
       [   1,   29,  842,  891,  687,  846],
       [   1,   31,  259,  466, 1134,  249],
       ...,
       [ 225, 1074,  420,  379,  400,  198],
       [ 225, 1075,  456,  283,  887,  197],
       [ 225, 1213,  298,  852,  384,  825]])


In [353]:
doc_id_all = np.array(list(doc_dict.keys()))
test_set = []
for q in test_qry_set:
  for i in range(0, 794, 5):
    if i == 790:
      test_set.append([q, doc_id_all[i], doc_id_all[i + 1], doc_id_all[i + 2], doc_id_all[i + 3], 0])
    else:
      test_set.append([q, doc_id_all[i], doc_id_all[i + 1], doc_id_all[i + 2], doc_id_all[i + 3], doc_id_all[i + 4]])
doc_dict[0] = ''
test_set = np.array(test_set)

In [354]:
pprint(test_set)

array([[ 187,    2,    3,    4,    5,    6],
       [ 187,   12,   13,   14,   15,   16],
       [ 187,   19,   20,   21,   22,   23],
       ...,
       [ 156, 1383, 1384, 1385, 1386, 1387],
       [ 156, 1391, 1392, 1393, 1394, 1395],
       [ 156, 1397, 1398, 1399, 1400,    0]])


In [355]:
class DataGeneration(tf.keras.utils.Sequence):
  
  def __init__(self, dataset, batch_size, wh_model, shuffle=True):
    self.data = dataset
    self.data_size = len(self.data)
    self.indexes = np.arange(self.data_size)
    self.batch_size = batch_size
    self.wh_model = wh_model
    self.shuffle = shuffle
  
  def __getitem__(self, idx):
    # get data in batch
    last_index = (idx + 1) * self.batch_size
    if last_index < self.data_size:
      indexes = self.indexes[idx * self.batch_size:last_index]
    else:
      indexes = self.indexes[idx * self.batch_size:]
    batch_data = self.data[indexes]
    # print(batch_data)
    # use word hashing to embed data
    batch_data_embed = np.zeros((0, 6, wh_size))
    for i, row in enumerate(batch_data):
      # print(i," ",row)
      batch_data_embed = np.concatenate([batch_data_embed, tf.expand_dims(self.embed(row), axis=0)], axis=0)
    
    return batch_data_embed, tf.zeros((batch_data_embed.shape[0], 1))  # y_true is useless in the model

  def __len__(self):
    return self.data_size // self.batch_size + 1

  def on_epoch_end(self):
    if self.shuffle:
      np.random.RandomState(500).shuffle(self.indexes)

  def embed(self, row):
    
    
    row_list = [qry_dict[row[0]]]
    # print(row_list)
    for i in range(1, 6):
      row_list.append(doc_dict[row[i]])
    return self.wh_model.transform(row_list).toarray()

In [356]:
train = DataGeneration(train_set, batch_size=32, wh_model=word_hashing)
test = DataGeneration(test_set, batch_size=32, wh_model=word_hashing)


In [357]:
print(train[0][0].shape)
print(train[0][0].shape)


(32, 6, 8501)
(32, 6, 8501)


In [358]:
# define cosine similarity layer
class CosSim(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
  
  def __call__(self, a, b):
    dot_product = tf.multiply(tf.math.l2_normalize(a, axis=1), tf.math.l2_normalize(b, axis=1))
    return tf.math.reduce_sum(dot_product, axis=1, keepdims=True)

In [359]:
# define DSSM model
class DSSM(tf.keras.Model):
  
  def __init__(self):
    super().__init__()
    self.fcn = tf.keras.Sequential(
        [tf.keras.layers.Dense(300, activation='tanh'),
         tf.keras.layers.Dense(300, activation='tanh'),
         tf.keras.layers.Dense(128, activation='tanh')
         
         ])
    self.sim = CosSim()
    self.concat = tf.keras.layers.Concatenate(axis=-1)

  def call(self, x):
    # fully connected layers
    q = self.fcn(x[:, 0, :])
    d = self.fcn(x[:, 1, :])
    d1 = self.fcn(x[:, 2, :])
    d2 = self.fcn(x[:, 3, :])
    d3 = self.fcn(x[:, 4, :])
    d4 = self.fcn(x[:, 5, :])
    # cosine similarity
    return self.concat([self.sim(q, d), self.sim(q, d1), self.sim(q, d2), self.sim(q, d3), self.sim(q, d4)])

In [361]:
# define the loss of DSSM
class DSSMLoss(tf.keras.losses.Loss):
  
  def call(self, y_true, y_pred):
    return -tf.math.log(tf.nn.softmax(y_pred, axis=-1)[:, 0])

# define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    verbose=1)

In [364]:
model = DSSM()
model.compile(loss=DSSMLoss(), optimizer=tf.optimizers.Adam(learning_rate=0.0005))

In [365]:
model.fit(train, epochs=20, callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 8s 225ms/step - loss: 1.6004
Epoch 2/20
28/28 [==============================] - 10s 374ms/step - loss: 1.2275
Epoch 3/20
28/28 [==============================] - 7s 232ms/step - loss: 1.0515
Epoch 4/20
28/28 [==============================] - 7s 250ms/step - loss: 0.9800
Epoch 5/20
28/28 [==============================] - 7s 263ms/step - loss: 0.9454
Epoch 6/20
28/28 [==============================] - 6s 227ms/step - loss: 0.9221
Epoch 7/20
28/28 [==============================] - 8s 290ms/step - loss: 0.9050
Epoch 8/20
28/28 [==============================] - 6s 225ms/step - loss: 0.8945
Epoch 9/20
28/28 [==============================] - 8s 282ms/step - loss: 0.8836
Epoch 10/20
28/28 [==============================] - 8s 278ms/step - loss: 0.8801
Epoch 11/20
28/28 [==============================] - 6s 223ms/step - loss: 0.8748
Epoch 12/20
28/28 [==============================] - 7s 226ms/step - loss: 0.8679
Epoch 13/20
28/28 [=====

In [366]:
test_pred = model.predict(test)

70/70 [==============================] - 16s 218ms/step


In [367]:
test_pred

array([[-0.40972787, -0.30119526, -0.2633507 ,  0.35643816, -0.455033  ],
       [-0.18512891, -0.36305946, -0.36003223, -0.35923648, -0.1860195 ],
       [ 0.6846875 , -0.22197965, -0.16595773, -0.38267887, -0.26574323],
       ...,
       [-0.6465901 , -0.6994611 , -0.73132026,  0.42512578,  0.05461963],
       [ 0.14483318,  0.03791783,  0.4882235 ,  0.14992163,  0.15362827],
       [-0.21282682, -0.06204201,  0.116076  ,  0.08540961, -0.05545541]],
      dtype=float32)

In [368]:
# define average precision (AP)
def avg_precision(retrieved_docs, relevant_docs):
  tp, fp, sum_precision = 0, 0, 0
  for i, doc in enumerate(retrieved_docs):
    if doc in relevant_docs:
      tp += 1
      sum_precision += tp / (tp + fp)
    else:
      fp += 1
  return sum_precision / len(retrieved_docs)

In [369]:
import numpy as np

def ndcg_score(retrieved_docs, relevant_docs, k=None):
    
    if k is None:
        k = len(retrieved_docs)
    k = min(k, len(retrieved_docs))

    # Calculate the DCG (Discounted Cumulative Gain)
    dcg = 0.0
    for i in range(k):
        doc_id = retrieved_docs[i]
        if doc_id in relevant_docs:
            rel = 1.0
        else:
            rel = 0.0
        dcg += (2 ** rel - 1) / np.log2(i + 2)

    # Calculate the IDCG (Ideal Discounted Cumulative Gain)
    ideal_retrieved_docs = sorted(relevant_docs, reverse=True)
    idcg = 0.0
    for i in range(k):
        if i >= len(ideal_retrieved_docs):
            break
        rel = 1.0
        idcg += (2 ** rel - 1) / np.log2(i + 2)

    # Calculate the nDCG score
    if idcg == 0:
        return 0.0
    else:
        return dcg / idcg


In [370]:
# define reciprocal rank (RR)
def reciprocal_rank(retrieved_docs, relevant_docs):
  rr = 0
  for i, doc in enumerate(retrieved_docs):
    if doc in relevant_docs:
      rr = 1 / (i + 1)
      break
  return rr

In [372]:
ap, rr,nd = 0, 0,0
test_seg = test_pred.shape[0] // len(test_qry_set)
for i, q in enumerate(test_qry_set):
  test_pred_q = test_pred[i * test_seg:(i + 1) * test_seg].flatten()[:1162]  # all predictions for each query
  retrieved_10_idx = np.argsort(test_pred_q)[::-1][:10]  # reverse sort and get top-10 index
  # print(retrieved_10_idx)
  # print(rel_dict[q])
  ap += avg_precision(doc_id_all[retrieved_10_idx], rel_dict[q])
  rr += reciprocal_rank(doc_id_all[retrieved_10_idx], rel_dict[q])
  nd += ndcg_score(doc_id_all[retrieved_10_idx], rel_dict[q])
print(f'MAP@10: {ap / len(test_qry_set)}')
print(f'MRR@10: {rr / len(test_qry_set)}')
print(f'NDCG: {nd / len(test_qry_set)}')

MAP@10: 0.013095238095238094
MRR@10: 0.07142857142857142
NDCG: 0.027223882437287145
